In [2]:
import cv2
import numpy as np
from scipy.signal import convolve2d
from scipy.ndimage.filters import correlate 
import matplotlib.pyplot as plt
import matplotlib.image as pltimg 
import math
import skimage
from scipy.ndimage.filters import gaussian_laplace
from scipy.ndimage.filters import maximum_filter

In [3]:
def load_image(path):
    '''
    :param path: path to load image
    :return: grayscaled and float image
    '''
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    # transmit the image to float
    img = skimage.img_as_float(img)
    print(img.dtype)
    return img

In [4]:
def normalize(img):
    ''' Function to normalize an input array to 0-1 '''
    img_min = img.min()
    img_max = img.max()
    return (img - img_min) / (img_max - img_min)

def generate_laplacian_filter(size, sigma):
    return gaussian_laplace(np.zeros((size,size)),sigma=sigma)

In [5]:
def show_all_circles(image, cx, cy, rad, color='r'):
    """
    image: numpy array, representing the grayscsale image
    cx, cy: numpy arrays or lists, centers of the detected blobs
    rad: numpy array or list, radius of the detected blobs
    """
    import matplotlib.pyplot as plt
    from matplotlib.patches import Circle

    fig, ax = plt.subplots()
    ax.set_aspect('equal')
    ax.imshow(image, cmap='gray')
    for x, y, r in zip(cx, cy, rad):
        circ = Circle((x, y), r, color=color, fill=False)
        ax.add_patch(circ)

    plt.title('%i circles' % len(cx))
    plt.show()

In [6]:
# define parameters
'''
k           increasing factor K
levels      number of iterations
ini_sigma   scale for LOG
'''
threshold = 0.25              
k = 1.25                     
levels = 12                   
sigma = 2

BUTTERFLY = "data\\part2\\butterfly.jpg" 
EINSTEIN = "data\\part2\\einstein.jpg"
FISHES = "data\\part2\\fishes.jpg"
SUNFLOWERS =  "data\\part2\\sunflowers.jpg"

In [59]:
# upscaling


def upscaling_method(img, sigma, k, levels, threshold):
  
    
    initial_sigma = sigma
    height, width = img.shape
    img_scale_space = np.zeros((height, width, levels), dtype=np.float64)
    
    for i in range(levels):
        # element wise scaling
        img_scale_space[:, :, i] = np.power(gaussian_laplace(img, sigma=sigma) * sigma**2, 2)
        sigma = sigma * k
    
    
    suppressed_space = np.zeros(img_scale_space.shape, dtype=np.float64)
    
    # nonmaximum supression in each 2d slice
    for i in range(levels):
        suppressed_space[:, :, i] = maximum_filter(suppressed_space[:, :, i], size=3)
    
    
    maxima_space = np.zeros((height, width), dtype=np.float64)
    # along with level max value
    maxima_space = np.argmax(suppressed_space, axis=2)
    survive_space = np.zeros(img_scale_space.shape)
    
    print("maxima space:")
    print(maxima_space)
    # locating survive pos
    for i in range(levels):
        survive_space[:, :, i] = (maxima_space == img_scale_space)
        survive_space[:, :, i] = survive_space[:, :, i] * img
    
    cx = []
    cy = []
    radius = []
    for i in range(levels):
        location = np.argwhere(survive_space[:, :, i] >= threshold)
        rad = np.sqrt(2) * initial_sigma * np.power(k,i)
        cx += [x[0] for x in location]
        cy += [x[1] for x in location]
        radius += rad * np.ones((len(location),1))
    
    show_all_circles(img,cx,cy,radius)
    

In [60]:
# downsampling


In [61]:
img = load_image(BUTTERFLY)
upscaling_method(img, sigma, k, levels, threshold)

float64


[[[6.11404630e-04 9.13829416e-04 1.02442151e-03 ... 4.18814176e-05
   4.99533358e-04 6.32839405e-03]
  [3.93883428e-04 5.78375614e-04 6.82894084e-04 ... 3.97897226e-05
   5.09108168e-04 6.35185348e-03]
  [1.31810465e-04 1.85957009e-04 2.55355303e-04 ... 3.57650309e-05
   5.28483029e-04 6.39717618e-03]
  ...
  [7.43311392e-05 2.25514363e-04 5.55462860e-04 ... 3.05732601e-04
   9.06283558e-04 1.63209232e-03]
  [1.72663844e-07 3.54195933e-05 2.78791064e-04 ... 3.26208473e-04
   9.32166422e-04 1.66086454e-03]
  [3.66565581e-05 8.52129420e-09 1.52739932e-04 ... 3.36803522e-04
   9.45302577e-04 1.67535800e-03]]

 [[6.28444882e-04 8.81459237e-04 9.94236384e-04 ... 3.92058807e-05
   5.09078097e-04 6.35019923e-03]
  [3.59358107e-04 5.37574172e-04 6.56036083e-04 ... 3.71884180e-05
   5.18722586e-04 6.37366031e-03]
  [8.03715156e-05 1.51922411e-04 2.36333410e-04 ... 3.33113595e-05
   5.38235298e-04 6.41898561e-03]
  ...
  [1.23046681e-04 2.59661096e-04 5.31007619e-04 ... 3.20010590e-04
   9.17684

C:\Users\liuzhe\mp0\lib\site-packages\ipykernel_launcher.py:33: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
